<a href="https://colab.research.google.com/github/vitroid/gromacs-usecases/blob/main/MDAnalysis%2BGromacs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colabには標準でMDAnalysisはインストールされていないので各自でインストールする。

In [ ]:
pip install mdanalysis

デモ用に、氷の構造をGenIceで生成する。

In [ ]:
! pip install genice2
! genice2 CS1 -r 2 2 2 > cs1.gro

さて、さっそくgroを読みこませてみる。やっぱり`.top`は読んでくれないので、`.gro`だけ読む。

In [ ]:
import MDAnalysis as mda

universe = mda.Universe("cs1.gro")
print(universe)

読めました。

yaplotに変換するコードを書いてみよう。

In [ ]:
%pip install yaplotlib

In [ ]:
import MDAnalysis as mda
import yaplotlib as yap

def main():
    u = mda.Universe("cs1.gro") # 事前にopenしなくてもファイル名だけでいい。
    for residue in u.residues:
        if residue.resname[:3] == "SOL":
            C = None
            for atom in residue.atoms:
                if atom.name == "O":
                    O = u.atoms.positions[atom.index]
                elif atom.name == "H":
                    print(yap.Line(u.atoms.positions[atom.index], O), end="")

main()

Profilerで実行時間を測る。

In [ ]:
import profile

profile.run('main()')

ざっくりと、水のケージ解析を書いてみよう。

In [ ]:
%pip install cycless pairlist

In [ ]:
import MDAnalysis as mda
from cycless.cycles import cycles_iter
from cycless.polyhed import polyhedra_iter
import pairlist
import numpy as np
import networkx as nx

def water_HB_digraph(waters, cellmat):
    dg = nx.DiGraph()
    celli   = np.linalg.inv(cellmat)
    H = np.array([u.atoms.positions[atom.index] for water in waters for atom in water.atoms if atom.name == "H"], dtype=float)
    O = np.array([u.atoms.positions[atom.index] for water in waters for atom in water.atoms if atom.name == "O"], dtype=float)
    # In a fractional coordinate
    rH = H @ celli
    rO = O @ celli
    # O-H distance is closer than 2.45 AA
    # Matsumoto JCP 2007 https://doi.org/10.1063/1.2431168
    for i,j,d in pairlist.pairs_iter(rH, 2.45, cellmat, pos2=rO):
        # but distance is greater than 1 AA (i.e. O and H are not in the same molecule)
        if 1 < d:
            # label of the molecule where Hydrogen i belongs.
            imol = i // 2
            # H to O vector
            # vec attribute is useful when you use cycless.dicycles.
            dg.add_edge(imol, j, vec=rO[j] - rH[i])
    return dg, rO


# Unfortunately, MDAnalysis does not read the concatenated gro file.
# https://docs.mdanalysis.org/stable/documentation_pages/coordinates/GRO.html
traj = open("cs1.gro")

u = mda.Universe(traj)
# cell dimension a,b,c,A,B,G
# Note: length unit of MDAnalysis is AA, not nm.
dimen   = u.trajectory.ts.dimensions
# cell matrix (might be transposed)
cellmat = mda.lib.mdamath.triclinic_vectors(dimen)
# Pick up water molecules only
waters = [residue for residue in u.residues if residue.resname[:3] == "SOL"]

# make a graph of hydrogen bonds and fractional coordinate of its vertices
dg, rO = water_HB_digraph(waters, cellmat)
# undirected graph
g = nx.Graph(dg)
# detect the pentagons and hexagons.
cycles = [cycle for cycle in cycles_iter(g, maxsize=6, pos=rO) if len(cycle) > 4]
# detect the cages with number of faces between 12 and 16.
cages  = [cage for cage in polyhedra_iter(cycles, maxnfaces=16) if len(cage) > 11]
for cage in cages:
    print(len(cage), cage)

こういう解析をするなら特段便利でもない。でも、MDAnalysisで使えるようにコードを作っておくと、他人と共有する時に手間が減ると思われる。

ここでは1フレームのみのgroファイルを読んだが、複数のフレームが含まれている時はこの書き方では最初のフレームしか読まない。MDAnalysisは、複数のフレームが含まれるgroファイルを読む機能がない。https://docs.mdanalysis.org/stable/documentation_pages/coordinates/GRO.html

複数のフレームを連続的に読みこむためのハックはこちら。

In [ ]:
import MDAnalysis as mda
import io


def gro_iter(file):
    """.groファイルに含まれる複数の座標セットを、別々のファイルとして提供する。

    Args:
        file (file handle): file handle of a .gro file containing multiple frames.
    """
    s = []
    Natom = -99
    for line in file:
        s.append(line)
        if len(s) == 2:
            Natom = int(line)
        elif len(s) == Natom+3:
            yield io.StringIO("".join(s))
            s = []
            Natom = -99


with open("trajectory.gro") as f:
    for file in gro_iter(f):
        u = mda.Universe(file, format="GRO")
        # ...

書きだすほうはもっと不便で、MDAnalysisのWriterはファイル名指定による出力しかできないので、書き方を自分で制御できない。例えば、文字列に結果を入れたり、file handleを使ってstdoutに書きだしたりすることもままならない。このあたりはセンスが悪い。

もうちょっと直接的にMDAnalysisを使う方法は? 例えば動径分布関数(RDF)ぐらいはすぐ出せるだろうか。


In [ ]:
import MDAnalysis as mda
import MDAnalysis.analysis.rdf as RDF
from matplotlib import pyplot as plt

u = mda.Universe("cs1.gro")

# Select carbons
# https://docs.mdanalysis.org/stable/documentation_pages/selections.html
O = u.select_atoms("resname SOL and name O")

# Make RDF class
# https://docs.mdanalysis.org/stable/documentation_pages/analysis/rdf.html
rdf = RDF.InterRDF(O, O)
rdf.run()

fig = plt.figure()
plt.ylim(0,3)
plt.xlim(1, 15)
plt.plot(rdf.bins, rdf.rdf)
fig.savefig("OO.pdf")

py3DMolを使い、Colab上で作画してみる。

In [ ]:
%pip install py3Dmol

In [ ]:
import MDAnalysis as mda
import pairlist
import numpy as np
import networkx as nx

def water_HB_digraph(waters, cellmat):
    dg = nx.DiGraph()
    celli   = np.linalg.inv(cellmat)
    H = np.array([u.atoms.positions[atom.index] for water in waters for atom in water.atoms if atom.name == "H"], dtype=float)
    O = np.array([u.atoms.positions[atom.index] for water in waters for atom in water.atoms if atom.name == "O"], dtype=float)
    # In a fractional coordinate
    rH = H @ celli
    rO = O @ celli
    # O-H distance is closer than 2.45 AA
    # Matsumoto JCP 2007 https://doi.org/10.1063/1.2431168
    for i,j,d in pairlist.pairs_iter(rH, 2.45, cellmat, pos2=rO):
        # but distance is greater than 1 AA (i.e. O and H are not in the same molecule)
        if 1 < d:
            # label of the molecule where Hydrogen i belongs.
            imol = i // 2
            # H to O vector
            # vec attribute is useful when you use cycless.dicycles.
            dg.add_edge(imol, j, vec=rO[j] - rH[i])
    return dg, rO


# Unfortunately, MDAnalysis does not read the concatenated gro file.
# https://docs.mdanalysis.org/stable/documentation_pages/coordinates/GRO.html
traj = open("cs1.gro")

u = mda.Universe(traj)
# cell dimension a,b,c,A,B,G
# Note: length unit of MDAnalysis is AA, not nm.
dimen   = u.trajectory.ts.dimensions
# cell matrix (might be transposed)
cellmat = mda.lib.mdamath.triclinic_vectors(dimen)
# Pick up water molecules only
waters = [residue for residue in u.residues if residue.resname[:3] == "SOL"]

# make a graph of hydrogen bonds and fractional coordinate of its vertices
dg, rO = water_HB_digraph(waters, cellmat)

# centering
rO -= 0.5

In [ ]:

import py3Dmol

def a2v3(a, labels="xyz"):
    return dict(zip(labels,a))

def Line(view,a,b,width=1.0):
    # https://3dmol.csb.pitt.edu/doc/$3Dmol.GLShape.html#addLine
    view.addLine({"start": a2v3(a),
                         "end":   a2v3(b),
                    # "color":"cyan",
                    })

def drawEdges(view, dg, rO, cellmat):
    for i,j in dg.edges():
        d = rO[j] - rO[i]
        d -= np.floor(d+0.5)
        a = rO[i] @ cellmat
        b = (rO[i]+d) @ cellmat
        Line(view, a, b)

view = py3Dmol.view()
drawEdges(view, dg, rO, cellmat)
view
